# 1. Tensor basic

In [129]:
import torch 

## Operation

In [18]:
x = torch.empty(1)
x

tensor([6.])

In [24]:
x = 2 * torch.ones(1, 10)
x

tensor([[2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]])

In [25]:
y = torch.ones_like(x)
y

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [27]:
y.add(x)
print(y)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [28]:
y.add_(x)
print(y)

tensor([[3., 3., 3., 3., 3., 3., 3., 3., 3., 3.]])


In [29]:
y.view(2, 5)

tensor([[3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.]])

## Numpy and pytorch

In [30]:
import torch
import numpy as np

In [36]:
a = torch.ones(5)
print(type(a))

b = a.numpy()
print(type(b))

<class 'torch.Tensor'>
<class 'numpy.ndarray'>


In [40]:
# Memory address of the tensor data: .data_ptr()
# Memory address of the numpy data: .ctypes.data

a.data_ptr() == b.ctypes.data

True

## Pytorch / Numpy device

In [46]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(True)
else:
    device = torch.device('cpu')
    print(False)

False


In [60]:
a = torch.ones(5, device= device)
a.to('cpu')
type(a)

torch.Tensor

In [58]:
b = a.numpy()
b.to_device('cpu')
type(b)

numpy.ndarray

# 2. Gradient auto 

## Gradient calculation

In [2]:
x = torch.ones(3, requires_grad= True)
print(x)

tensor([1., 1., 1.], requires_grad=True)


In [3]:
y = x + 2

In [4]:
print(y)

tensor([3., 3., 3.], grad_fn=<AddBackward0>)


### With scalar

In [5]:
z = y*y*2
z = z.mean()
print(z)

tensor(18., grad_fn=<MeanBackward0>)


In [6]:
z.backward() # dz/dx

In [7]:
x.grad

tensor([4., 4., 4.])

### With vector

In [8]:
x = torch.ones(3, requires_grad= True)
y = x + 2

f = y*y*2
# vector = torch.tensor([0.1, 0.01, 0.001])
vector = torch.ones(3)
print(f)

tensor([18., 18., 18.], grad_fn=<MulBackward0>)


In [9]:
f.backward(vector) # df/fx

In [10]:
x.grad

tensor([12., 12., 12.])

## Preventing gradient history

In [11]:
x = torch.ones(3, requires_grad= True)
# x.requires_grad_(False)
# x.detach() # create new tensor 

In [12]:
with torch.no_grad():
    y = x + 2
    print(y)

tensor([3., 3., 3.])


## Note: gradient for tensor which has required grad will be accumulated  

In [13]:
weights = torch.ones(4, requires_grad= True)
for epoch in range(1):
    model_output = (weights*3).sum()
    model_output.backward()

    print(weights.grad)

tensor([3., 3., 3., 3.])


In [14]:
weights = torch.ones(4, requires_grad= True)
for epoch in range(2):
    model_output = (weights*3).sum()
    model_output.backward()

    print(weights.grad)

tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])


In [15]:
# fix grad to not accumulate

weights = torch.ones(4, requires_grad= True)
for epoch in range(2):
    model_output = (weights*3).sum()
    model_output.backward()
    
    print(weights.grad)
    weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


In [16]:
# with optimizer

# optimizer = torch.optim.SGD(params= weights, lr= 0.01)
# optimizer.step()
# optimizer.zero_grad()

# 3. Backpropagation 

The chain rules:
$$
y = f(x), \quad z = g(y)
$$
$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial y} \cdot \frac{\partial y}{\partial x}
$$




Example:
$$
\hat{y} = wx
$$

$$
s = \hat{y} - y
$$

$$
\text{Loss} = (s)^2
$$

$$
\frac{\partial \text{Loss}}{\partial w} = \frac{\partial \text{Loss}}{\partial s} \cdot \frac{\partial s}{\partial \hat{y}} \cdot \frac{\partial \hat{y}}{\partial w} = 2(wx-y)x
$$


In [70]:
x = torch.tensor(1.0)
y = torch.tensor(2.0)
w = torch.tensor(1.0, requires_grad= True)

In [71]:
y_hat = w*x 
loss = (y_hat - y)**2
loss

tensor(1., grad_fn=<PowBackward0>)

In [72]:
loss.backward()
w.grad

tensor(-2.)

# 4.Gradient Descent with Autograd and Backpropagation

##  Numpy

In [37]:
import numpy as np

In [50]:
import numpy as np

# Input data
X = np.array([1, 2, 3, 4], dtype=np.float64)
y = np.array([1, 4, 2, 1], dtype=np.float64)

# Initialize weight to zero
w = 0.0 

# Model prediction
def forward(x):
    return w * x

# Loss function (MSE)
def loss(y, y_pred):
    return ((y - y_pred)**2).mean()

# Gradient calculation
# dL/dw = -2/N * sum(x * (y - y_pred))
def gradient(x, y, y_pred):
    return np.dot(2*x, y-y_pred).mean()

print(f'Prediction before training f(5) = {forward(5):.3f}')

# Training parameters
learning_rate = 0.001
n_iter = 10

# Training loop
for epoch in range(n_iter):
    # Predictions
    y_pred = forward(X)

    # Compute loss
    l = loss(y, y_pred)

    # Compute gradient
    dL_dw = gradient(X, y, y_pred)

    # Update weights
    w -= learning_rate * dL_dw

    # Print progress
    if (epoch + 1 % 100) == 0:
        print(f'Epoch: {epoch + 1}, w = {w:.3f}, loss = {l:.3f}')

# Prediction after training
print(f'Prediction after training f(5) = {forward(5):.3f}')

Prediction before training f(5) = 0.000
Prediction after training f(5) = -2.504


## Pytorch

In [42]:
import torch 

In [44]:
# f = w * x

X = torch.tensor([1, 2, 3, 4], dtype= torch.float64)
y = torch.tensor([1, 4, 2, 1], dtype= torch.float64)

# W = 0 
w = torch.tensor(0, dtype= torch.float64, requires_grad= True)

# model prediction
def forward(x):
    return w * X

# loss = MSE
def loss(y, y_pred):
    return ((y - y_pred)**2).mean()

# gradient 
# MSE = 1/N * (y - y_pred)**2
# dJ/dw = 1/N * (y - y_pred)**2 * 2w
def gradient(x, y, y_pred):
    return torch.dot(2*x, y - y_pred).mean()

print(f'Prediction before training f(5)= {forward(5)}') 

learning_rate = 1e-6
n_iter = 1000

for epoch in range(n_iter):
    # predic
    y_pred = forward(X)
    l = loss(y, y_pred) 

    l.backward()
    # dL_dw = gradient(X, y, y_pred)

    with torch.no_grad():
        w -= learning_rate*w.grad

    # zero gradient
    w.grad.zero_()

    if (epoch + 1) % 1000 == 0:
        print(f'Epoch: {epoch + 1}, w= {w:.3f}, loss= {l:.3f}')

print(f'prediction: {forward(5)}')

Prediction before training f(5)= tensor([0., 0., 0., 0.], dtype=torch.float64, grad_fn=<MulBackward0>)
Epoch: 1000, w= 0.009, loss= 5.411
prediction: tensor([0.0094, 0.0189, 0.0283, 0.0377], dtype=torch.float64,
       grad_fn=<MulBackward0>)


# 5. Training Pipeline: Model, Loss, and Optimizer

1. Design model (input, output, forward pass)
2. Construct loss and optimizer
3. Training loop:
    - Forward pass: compute prediction
    - Backward pass: gradients
    - Update weights

In [128]:
import torch 
import torch.nn as nn

# f = w * x

X = torch.tensor([1, 2, 3, 4], dtype= torch.float64).reshape(4, 1)
y = torch.tensor([1, 4, 2, 1], dtype= torch.float64).reshape(4, 1)
X_test = torch.tensor([10], dtype= torch.float64)

n_samples, n_feature = X.shape

# # W = 0 
# w = torch.tensor(0, dtype= torch.float64, requires_grad= True)

# # model prediction
# def forward(x):
#     return w * X
model = nn.Linear(in_features= n_feature, out_features= n_feature, dtype= torch.float64) # Cácch siêu đơn giản 

class LinearRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear_stack = nn.Linear(
            in_features= input_dim,
            out_features= output_dim,
            dtype= torch.float64
        )

    def forward(self, x):
        return self.linear_stack(x)
    
model = LinearRegression(input_dim= n_feature, output_dim= n_feature)

learning_rate = 1e-6
n_iter = 1

# loss = MSE
# def loss(y, y_pred):
#     return ((y - y_pred)**2).mean()
loss = nn.MSELoss()

# optimizer = torch.optim.SGD(params= [w], lr= learning_rate)
optimizer = torch.optim.SGD(params= model.parameters(), lr= learning_rate)

# gradient 
# MSE = 1/N * (y - y_pred)**2
# dJ/dw = 1/N * (y - y_pred)**2 * 2w    
# def gradient(x, y, y_pred):
#     return torch.dot(2*x, y - y_pred).mean()

# Chỗ này cần chỉnh lại vì không còn hàm forward như trước nữa
# Chúng ta đã sử dụng model, nên phần này có thể sửa lại để gọi model(X_test) cho dự đoán
print(f'Prediction before training f(5)= {model(torch.tensor([[5.0]], dtype=torch.float64))}') 

for epoch in range(n_iter):
    # predict
    # y_pred = forward(X)
    y_pred = model(X)

    l = loss(y, y_pred) 

    # dL_dw = gradient(X, y, y_pred)
    l.backward()

    # with torch.no_grad():
    #     w -= learning_rate*w.grad
    optimizer.step()

    # zero gradient
    # w.grad.zero_()
    optimizer.zero_grad()

    # Điều kiện in kết quả chỉ mỗi 1000 epoch
    
    [w, b] = model.parameters()
    print(f'[w, b] = [{w.item()}, {b.item()}]')
    # print(f'Epoch: {epoch + 1}, w= {w.data}, loss= {l.item():.3f}')

# Dự đoán sau khi huấn luyện
print(f'Prediction {model(X_test)}')
# print(f'prediction: {forward(5)}')


Prediction before training f(5)= tensor([[3.6725]], dtype=torch.float64, grad_fn=<AddmmBackward0>)
[w, b] = [0.9039510325745299, -0.847219850869349]
Prediction tensor([8.1923], dtype=torch.float64, grad_fn=<ViewBackward0>)


### Giải thích từ Chat GPT

Giải thích:
Sử dụng model(X) vs model.forward(X):

Trong PyTorch, bạn luôn nên sử dụng model(X) thay vì gọi trực tiếp model.forward(X). Lý do là PyTorch sử dụng phương thức __call__() bên trong, phương thức này bao gồm nhiều chức năng bổ sung như hooks và đảm bảo hành vi chính xác trong quá trình huấn luyện và đánh giá (chẳng hạn như khi sử dụng dropout hoặc batch normalization).
Bằng cách gọi model(X), PyTorch sẽ tự động gọi model.forward(X) bên trong và xử lý các thao tác bổ sung như hooks và các chế độ (train/eval).
Gọi trực tiếp forward():

Mặc dù có thể dễ dàng gọi trực tiếp forward() bằng cách viết model.forward(X), điều này sẽ bỏ qua các tính năng bổ sung (chẳng hạn như các hooks đã đăng ký hoặc ngữ cảnh torch.no_grad()). Do đó, không khuyến khích việc gọi forward() trực tiếp trừ khi bạn muốn bỏ qua các tính năng bổ sung, điều này hiếm khi cần thiết.

### Test param 
https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html#torch.nn.parameter.Parameter

In [102]:
w


Parameter containing:
tensor([[0.4039]], dtype=torch.float64, requires_grad=True)

In [103]:
type(w)

torch.nn.parameter.Parameter

In [105]:
w[0]


tensor([0.4039], dtype=torch.float64, grad_fn=<SelectBackward0>)

In [106]:
type(w[0])

torch.Tensor

In [107]:
w[0][0]


tensor(0.4039, dtype=torch.float64, grad_fn=<SelectBackward0>)

In [108]:
type(w[0][0])

torch.Tensor

In [109]:
w[0][0].item()


0.40389969568636214

In [120]:
w.item()

0.40389969568636214

In [110]:
type(w[0][0].item())

float

In [116]:
b

Parameter containing:
tensor([-0.0690], dtype=torch.float64, requires_grad=True)

In [117]:
type(b)

torch.nn.parameter.Parameter

In [118]:
b[0]

tensor(-0.0690, dtype=torch.float64, grad_fn=<SelectBackward0>)

In [119]:
type(b[0])

torch.Tensor

In [123]:
b.item()

-0.06901760123018814

# 6. Linear Regression

In [71]:
import torch
import torch.nn as nn
import numpy as np 
from sklearn import datasets
import plotly.graph_objects as go

## Prepare datasets

In [96]:
X_numpy, y_numpy = datasets.make_regression(n_samples= 100, n_features= 1, noise= 20, random_state= 1)

In [97]:
X = torch.from_numpy(X_numpy.astype(np.float64))
y = torch.from_numpy(y_numpy.astype(np.float64))

In [98]:
y = y.view(-1, 1)   

Có, y.view() và y.reshape() trong PyTorch đều được sử dụng để thay đổi hình dạng của một tensor, nhưng chúng khác nhau về cách xử lý bộ nhớ.

Khác biệt chính:
view():

view(): tạo ra một tensor mới với hình dạng mong muốn mà không thay đổi dữ liệu. Tuy nhiên, nó yêu cầu tensor gốc phải có bộ nhớ liên tục (tức là các phần tử được lưu trong một khối liên tục).
Nếu tensor không liên tục, bạn phải gọi tensor.contiguous() trước, để tạo một bản sao của dữ liệu trong bộ nhớ liên tục trước khi áp dụng view().

reshape(): cố gắng trả về một tensor mới với hình dạng mong muốn và có thể trả về một view nếu tensor đã liên tục. Tuy nhiên, nếu tensor không liên tục trong bộ nhớ, reshape() sẽ tự động tạo một bản sao của dữ liệu mà không yêu cầu gọi contiguous().
Điều này làm cho reshape() linh hoạt hơn nhưng có thể kém hiệu quả hơn trong một số trường hợp vì có thể liên quan đến việc sao chép dữ liệu.

## Design 

In [99]:
n_samples, n_feature = X.shape

In [100]:
# model 
input_size = n_feature
model = nn.Linear(in_features= input_size, out_features= 1, dtype= torch.float64)

##  Loss_fn and optimizer

In [101]:
criterion_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr= 0.3)

## Training loop


In [102]:
epochs = 1000
for epoch in range(epochs):
    y_pred = model(X)

    loss = criterion_fn(y_pred, y)

    loss.backward()
    
    optimizer.step()

    optimizer.zero_grad()

    if (epoch + 1) % 100 == 0:
        print(f'Epcoh: {epoch + 1}, loss= {loss.item(): .2f}')

with torch.no_grad():
    predicted = model(X).detach().numpy() 


Epcoh: 100, loss=  332.57
Epcoh: 200, loss=  332.57
Epcoh: 300, loss=  332.57
Epcoh: 400, loss=  332.57
Epcoh: 500, loss=  332.57
Epcoh: 600, loss=  332.57
Epcoh: 700, loss=  332.57
Epcoh: 800, loss=  332.57
Epcoh: 900, loss=  332.57
Epcoh: 1000, loss=  332.57


In [105]:
fig = go.Figure()
fig.add_trace(go.Scatter(x= X.reshape(-1,), y= y.reshape(-1,), mode= 'markers'))
fig.add_trace(go.Scatter(x= X.reshape(-1,), y= predicted.reshape(-1,), mode= 'lines'))
fig.show()

# 7. Logistic Regression

In [162]:
import numpy as np

import torch
import torch.nn as nn

import sklearn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Prepare datasets

In [163]:
# Load dataset

breast_cancer_dataset = datasets.load_breast_cancer()

In [164]:
# Infor of dataset

breast_cancer_dataset

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

In [165]:
X, y = breast_cancer_dataset['data'], breast_cancer_dataset['target']

In [166]:
n_samples, n_feature = X.shape

In [167]:
# Split dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size= 0.8, random_state= 1, 
                                                     shuffle= True, stratify= y)    

In [168]:
# Scale dataset

scale = StandardScaler()

X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)

In [169]:
# Convert to tensor

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))

y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [170]:
y_train.shape # view shape of y_train

y_train = y_train.view(-1 , 1)
y_test = y_test.view(-1, 1)

## Define

In [185]:
class LogisticRegression(nn.Module):
    def __init__(self, X, y):
        super().__init__()
        self.n_samples, self.n_features = X.shape
        self.input = X
        self.output = y
        self.logistic_regression = nn.Sequential(
            nn.Linear(in_features= self.n_features, out_features= 1), # Dễ sai cái này, 30 input - > 0 or 1 nên out = 1
            nn.Sigmoid() 
        )

    def forward(self):
        return self.logistic_regression(self.input)

In [186]:
model = LogisticRegression(X= X_train, y= y_train)

In [187]:
X_train.shape

torch.Size([455, 30])

## Loss_fn and optimizer

In [188]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(params= model.parameters(), lr= 0.1)

# Training loop

In [190]:
epochs = 1000
for epoch in range(epochs):
    y_pred = model()

    loss = loss_fn(y_pred, y_train)

    loss.backward()

    optimizer.step()

    optimizer.zero_grad()

    if (epoch + 1) % 100 == 0:
        print(f'epoch: {epoch}, loss: {loss.item()}')



epoch: 99, loss: 0.06136834993958473
epoch: 199, loss: 0.060583651065826416
epoch: 299, loss: 0.05989560857415199
epoch: 399, loss: 0.05928619205951691
epoch: 499, loss: 0.0587414987385273
epoch: 599, loss: 0.05825095996260643
epoch: 699, loss: 0.05780624598264694
epoch: 799, loss: 0.05740055441856384
epoch: 899, loss: 0.05702850595116615
epoch: 999, loss: 0.056685686111450195
